In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
# LOAD DATAS 
seifa_data_path = 'C:/Users/TaeikCho/Desktop/Aus_pofol/seifa/datas/SA1-distribution-data-cubes-all/Statistical Area Level 2, SA1 Distributions, SEIFA 2021.xlsx'
df_dis = pd.read_excel(seifa_data_path, sheet_name='Table 1', header=5) #disadvantage
df_adv = pd.read_excel(seifa_data_path, sheet_name='Table 2', header=5) #advantage + disadvantage
df_ier = pd.read_excel(seifa_data_path, sheet_name='Table 3', header=5) #economic resource
df_ieo = pd.read_excel(seifa_data_path, sheet_name='Table 4', header=5) #employment and oppcupation 



In [5]:
# ORGANISE DISADVANTAGE DATA
df_dis = df_dis[['2021 Statistical Area Level 2  (SA2) 9-Digit Code',
       '2021 Statistical Area Level 2 (SA2) Name ',
       'Decile ranking of the SA2 within Australia']]
df_dis.columns = ['SA2_CODE', 'SA2_NAME', 'IRSD_DECILE']

# ORGANISE ADVANTAGE DATA 
df_adv = df_adv[['2021 Statistical Area Level 2  (SA2) 9-Digit Code',
       '2021 Statistical Area Level 2 (SA2) Name ',
       'Decile ranking of the SA2 within Australia']]
df_adv.columns = ['SA2_CODE', 'SA2_NAME', 'IRSAD_DECILE']

# ORGANISE ECONOMIC RESOURCE DATA 
df_ier = df_ier[['2021 Statistical Area Level 2  (SA2) 9-Digit Code',
       '2021 Statistical Area Level 2 (SA2) Name ',
       'Decile ranking of the SA2 within Australia']]
df_ier.columns = ['SA2_CODE', 'SA2_NAME', 'IER_DECILE']

# ORGANISE EMPLOYMENT AND OCCUPATION DATA 
df_ieo = df_ieo[['2021 Statistical Area Level 2  (SA2) 9-Digit Code',
       '2021 Statistical Area Level 2 (SA2) Name ',
       'Decile ranking of the SA2 within Australia']]

df_ieo.columns = ['SA2_CODE', 'SA2_NAME', 'IEO_DECILE']


In [6]:
from functools import reduce

# DATAFRAME LIST 
dfs = [df_adv, df_dis, df_ier, df_ieo]

# INNER / OUTER MERGE 
df_inner = reduce(lambda left, right: pd.merge(left, right, on=['SA2_CODE', 'SA2_NAME'], how='inner'), dfs)
df_outer = reduce(lambda left, right: pd.merge(left, right, on=['SA2_CODE', 'SA2_NAME'], how='outer'), dfs)

print(df_inner.equals(df_outer))

True


In [7]:
# GET MAP FILE (FORMAT : shp)
map_path = 'datas/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp'
gdf_map = gpd.read_file(map_path)

# EDIT GDF MAP  
gdf_map = gdf_map[['SA2_CODE21', 'GCC_NAME21']]
gdf_map.columns = ['SA2_CODE', 'GCC_NAME21']
gdf_map['SA2_CODE'] = gdf_map['SA2_CODE'].astype(str)

gdf_map


,SA2_CODE,GCC_NAME21
0,101021007,Rest of NSW
1,101021008,Rest of NSW
2,101021009,Rest of NSW
3,101021010,Rest of NSW
4,101021012,Rest of NSW
...,...,...
2468,901031003,Other Territories
2469,901041004,Other Territories
2470,997979799,Migratory - Offshore - Shipping (OT)
2471,999999499,No usual address (OT)


In [8]:
# MERGE SA2 AND GDF DATAS 
df_inner['SA2_CODE'] = df_inner['SA2_CODE'].astype(str)
final_map = gdf_map.merge(df_inner, on='SA2_CODE', how='left')

# SELECT GREATER SYDNEY
sydney = final_map[final_map['GCC_NAME21'] == 'Greater Sydney'].reset_index(drop=True)

# CREATE NEW COLUMNS : GAP BETWEEN IRSAD AND IRSD 
## if gap > 0 : there are fewer disadvantage than advantage
## if gap < 0 : there are more disadvantage thatn advantage 
sydney['DIS_ADV_GAP'] = sydney['IRSD_DECILE'] - sydney['IRSAD_DECILE']

# CHANGE ORDER OF COLUMNS 
sydney = sydney[['GCC_NAME21', 'SA2_CODE',  'SA2_NAME', 'IRSAD_DECILE', 'IRSD_DECILE', 'DIS_ADV_GAP','IER_DECILE', 'IEO_DECILE']]


sydney.head()

,GCC_NAME21,SA2_CODE,SA2_NAME,IRSAD_DECILE,IRSD_DECILE,DIS_ADV_GAP,IER_DECILE,IEO_DECILE
0,Greater Sydney,102011028,Avoca Beach - Copacabana,9.0,9.0,0.0,10.0,8.0
1,Greater Sydney,102011029,Box Head - MacMasters Beach,8.0,8.0,0.0,9.0,7.0
2,Greater Sydney,102011030,Calga - Kulnura,6.0,6.0,0.0,8.0,6.0
3,Greater Sydney,102011031,Erina - Green Point,7.0,7.0,0.0,8.0,6.0
4,Greater Sydney,102011032,Gosford - Springfield,5.0,4.0,-1.0,2.0,7.0


In [9]:
# CASE01 : all deciles are less than 3
cond_dis = sydney['IRSD_DECILE'] <= 1
cond_adv = sydney['IRSAD_DECILE'] <= 1
cond_ier = sydney['IER_DECILE'] <= 1
cond_ieo = sydney['IEO_DECILE'] <= 1

df_lower = sydney[cond_dis & cond_adv & cond_ier & cond_ieo]
print(f'all deciles are 1 : {len(df_lower)}')

# CASE02 : all deciles are 10 
cond_dis = sydney['IRSD_DECILE'] == 10
cond_adv = sydney['IRSAD_DECILE'] == 10
cond_ier = sydney['IER_DECILE'] == 10
cond_ieo = sydney['IEO_DECILE'] == 10

df_upper = sydney[cond_dis & cond_adv & cond_ier & cond_ieo]
print(f'all deciles are 10 : {len(df_upper)}')

# CASE03: GAPS are distictive
## CASE03-01 :: IRSAD > IRSD : [income or asset] is low but [education or employ] is high
cond_neggap = sydney['DIS_ADV_GAP'] <= -5
df_neg_gap = sydney[cond_neggap]
print(f'IRSAD > IRSD : {len(df_neg_gap)}')
## CASE03-02 :: IRSAD < IRSD : [income or asset] is high but [education or employ] is low
cond_posgap = sydney['DIS_ADV_GAP'] >= 2
df_pos_gap = sydney[cond_posgap]
print(f'IRSAD < IRSD : {len(df_pos_gap)}')

all deciles are 1 : 4
all deciles are 10 : 18
IRSAD > IRSD : 5
IRSAD < IRSD : 1


1. IRSD > IRSAD 
 - There are 32 suburbs where "gap" is one 
 - There are only one suburb where "gap" is two 
 - So, we can exclude this case 

2. IRSD < IRSAD 
 - there are 5 suburbs where gap is less than -4
 - So, we can discuss that suburbs

3. ALL 10 SUBURBS (IRSAD = IRSD = 10)
 - There are 18 suburbs with 10/10
 - We can extract 3~5 suburbs with some standars

4. ALL BAD SUBURBS (IRSAD = IRSD = IER = IEO = 1)
 - There are 4 suburbs with 1/1/1/1
 - We can discuss that suburbs

🚀 최종 작업 순서 (Workflow)

Step 1. 리스트 추출 (Pandas)
 - 3가지 그룹에 해당하는 Top 3 동네 이름 추출.

Step 2. 엑셀 검증 (Table 3 & 4)
 - 그 9개(3x3) 동네의 IER(경제력)과 IEO(교육/직업) 점수를 엑셀에서 확인.

Step 3. Census 팩트 체크 (Web Search)
 - Step 2에서 세운 가설이 맞는지 구글에 ABS Quickstats [동네이름]을 검색해서 확인.
 - 예시 : "돈이 없네?" -> Income 확인 / "학력이 높네?" -> Education 확인 / "학생인가?" -> Age 확인

Step 4. 리포트 작성
 - "High Gap 그룹인 Haymarket을 분석해보니, IRSD는 낮았지만 IEO(교육)는 10등급이었습니다. 실제 Census 확인 결과 대학생 비율이 56%였습니다." 라고 적으면 끝!

In [8]:
df_lower
df_upper
df_neg_gap

,GCC_NAME21,SA2_CODE,SA2_NAME,IRSAD_DECILE,IRSD_DECILE,DIS_ADV_GAP,IER_DECILE,IEO_DECILE
94,Greater Sydney,117031642,Redfern,9.0,4.0,-5.0,1.0,9.0
97,Greater Sydney,117031645,Sydney (South) - Haymarket,9.0,2.0,-7.0,1.0,9.0
98,Greater Sydney,117031646,Ultimo,8.0,3.0,-5.0,1.0,10.0
99,Greater Sydney,117031647,Waterloo,9.0,3.0,-6.0,1.0,9.0
180,Greater Sydney,120031678,Burwood (NSW),7.0,2.0,-5.0,1.0,9.0


In [9]:
# case 01 
df_lower

,GCC_NAME21,SA2_CODE,SA2_NAME,IRSAD_DECILE,IRSD_DECILE,DIS_ADV_GAP,IER_DECILE,IEO_DECILE
67,Greater Sydney,116031313,Bidwill - Hebersham - Emerton,1.0,1.0,0.0,1.0,1.0
70,Greater Sydney,116031316,Lethbridge Park - Tregear,1.0,1.0,0.0,1.0,1.0
274,Greater Sydney,124051581,St Marys - North St Marys,1.0,1.0,0.0,1.0,1.0
322,Greater Sydney,127011504,Ashcroft - Busby - Miller,1.0,1.0,0.0,1.0,1.0


In [10]:
df_upper

,GCC_NAME21,SA2_CODE,SA2_NAME,IRSAD_DECILE,IRSD_DECILE,DIS_ADV_GAP,IER_DECILE,IEO_DECILE
33,Greater Sydney,115011296,West Pennant Hills,10.0,10.0,0.0,10.0,10.0
38,Greater Sydney,115011557,Castle Hill - West,10.0,10.0,0.0,10.0,10.0
39,Greater Sydney,115011558,Cherrybrook,10.0,10.0,0.0,10.0,10.0
41,Greater Sydney,115011622,Kellyville - West,10.0,10.0,0.0,10.0,10.0
107,Greater Sydney,118011346,Rose Bay - Vaucluse - Watsons Ba,10.0,10.0,0.0,10.0,10.0
187,Greater Sydney,121011683,Castle Cove - Northbridge,10.0,10.0,0.0,10.0,10.0
189,Greater Sydney,121011685,Greenwich - Riverview,10.0,10.0,0.0,10.0,10.0
194,Greater Sydney,121021406,Normanhurst - Thornleigh - Westl,10.0,10.0,0.0,10.0,10.0
199,Greater Sydney,121031408,Lindfield - Roseville,10.0,10.0,0.0,10.0,10.0
200,Greater Sydney,121031409,Pymble,10.0,10.0,0.0,10.0,10.0


In [11]:
df_neg_gap

,GCC_NAME21,SA2_CODE,SA2_NAME,IRSAD_DECILE,IRSD_DECILE,DIS_ADV_GAP,IER_DECILE,IEO_DECILE
94,Greater Sydney,117031642,Redfern,9.0,4.0,-5.0,1.0,9.0
97,Greater Sydney,117031645,Sydney (South) - Haymarket,9.0,2.0,-7.0,1.0,9.0
98,Greater Sydney,117031646,Ultimo,8.0,3.0,-5.0,1.0,10.0
99,Greater Sydney,117031647,Waterloo,9.0,3.0,-6.0,1.0,9.0
180,Greater Sydney,120031678,Burwood (NSW),7.0,2.0,-5.0,1.0,9.0


In [197]:
cond1 = sydney['IRSD_DECILE'] == 10
cond2 = sydney['IRSAD_DECILE'] == 10
cond3 = sydney['IER_DECILE'] == 10
cond4 = sydney['IEO_DECILE'] == 10

all_ten = sydney[cond1 & cond2 & cond3 & cond4]
print(len(all_ten))

18


In [175]:
deciles = ['IRSAD_DECILE', 'IRSD_DECILE', 'IER_DECILE', 'IEO_DECILE']
q_list = [0.2, 0.15, 0.1, 0.05]
print(f'sydney datas : {len(sydney)}\n')

for q in q_list:
    for dec in deciles: 
        print(f'{dec} lower {q*100}%: {sydney[dec].quantile(q)}')
    print('')    

sydney datas : 373

IRSAD_DECILE lower 20.0%: 4.0
IRSD_DECILE lower 20.0%: 3.0
IER_DECILE lower 20.0%: 2.0
IEO_DECILE lower 20.0%: 5.0

IRSAD_DECILE lower 15.0%: 3.0
IRSD_DECILE lower 15.0%: 2.0
IER_DECILE lower 15.0%: 2.0
IEO_DECILE lower 15.0%: 4.0

IRSAD_DECILE lower 10.0%: 2.0
IRSD_DECILE lower 10.0%: 1.0
IER_DECILE lower 10.0%: 1.0
IEO_DECILE lower 10.0%: 3.0

IRSAD_DECILE lower 5.0%: 1.0
IRSD_DECILE lower 5.0%: 1.0
IER_DECILE lower 5.0%: 1.0
IEO_DECILE lower 5.0%: 2.0

